In [1]:
#imports

from statistics import mean
import torch
import numpy as np
import pyro
import pyro.distributions as dist
from pyro.infer import Importance, EmpiricalMarginal
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

pyro.set_rng_seed(101)

In [3]:
A_label = ['U21', 'U25', 'U30', 'A30']

N_label = ['AF','AS','EU','N_A','OC','SA' ]

P1_label = ['CAM','CB','CDM','CF','CM','GK','LB','LW','RB','RW']

OC_label = ['Tier1', 'Tier2', 'Tier3', 'Tier4']

OL_label = ['1 Bundesliga','Eredivisie','Liga Nos','Ligue 1', 'Other','Premier League','Premier Liga','Primera Division','Serie A' ]

AC_label = ['Tier1', 'Tier2', 'Tier3', 'Tier4']

AL_label = ['1 Bundesliga','Eredivisie','Liga Nos','Ligue 1','Premier League','Premier Liga','Primera Division','Serie A' ]

Ovr_label = ['below65','65to74','75to84','85above']

Pot_label = ['below65','65to74','75to84','85above']

Y1_label = ['2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019']

T1_label = ['Above80M','80Mto40M','40Mto10M','10Mto1M','Below1M']


In [ ]:
A_prob = torch.tensor([0.27103765, 0.41418855, 0.27182854, 0.04294527])

N_prob = torch.tensor([0.101655594, 0.027312032, 0.695296847, 0.017821365, 0.004059897, 0.153854266])

P1_prob = torch.tensor([0.06311294, 0.17494464, 0.08509965, 0.20563113, 0.12496046, 0.05963303, 0.06849098, 0.08256881, 0.06058209, 0.07497627])

OC_prob = torch.tensor([[0.2879670550, 0.3801727601, 0.1632181599, 0.1686420249], 
                        [0.0008986341, 0.2532350827, 0.3567577283, 0.3891085550],
                        [0.3699867783, 0.2033935654, 0.0962979286, 0.3303217276],
                        [0.2949854886, 0.2325862625, 0.2180748146, 0.2543534344],
                        [0.0001298566, 0.0001298566, 0.0001298566, 0.9996104301],
                        [0.5458933376, 0.2624624149, 0.1499446115, 0.0416996360],
                        [0.1271237864, 0.5421723301, 0.1762742718, 0.1544296117],
                        [0.4344590091 ,0.1565207732, 0.2724950011, 0.1365252166],
                        [0.4066726277, 0.3422935062, 0.1994523304, 0.0515815357]
                       ])


OL_prob = torch.tensor([0.08748989, 0.04889451, 0.03987838, 0.10900207, 0.33835987, 0.11105839, 0.02896411, 0.07910647, 0.15724630])

AC_prob = torch.tensor([[0.2353747715, 0.4725124053, 0.2071689736, 0.0849438496],
                        [0.0009520183, 0.2507616146, 0.4122239147, 0.3360624524],
                        [0.4571373157, 0.2243987587, 0.1219937936, 0.1964701319],
                        [0.2698696057, 0.2872555107, 0.3120925179, 0.1307823657],
                        [0.5149482358, 0.3662053666, 0.1067504754, 0.0120959222],
                        [0.1005986465, 0.5566111400, 0.1318323790, 0.2109578345],
                        [0.4591062395, 0.1663575042, 0.2985666105, 0.0759696459],
                        [0.3837876018, 0.4349759016, 0.1457952468, 0.0354412498]
                       ])

AL_prob = torch.tensor([0.15141569, 0.05192186, 0.05097279, 0.12737267, 0.18716387, 0.07596488, 0.11724929, 0.23793894])

Ovr_prob = torch.tensor([[0.6748120301, 0.2362155388, 0.0156641604, 0.0733082707],
                         [0.6837703436, 0.1910036166, 0.0065551537, 0.1186708861],
                         [0.6770446097, 0.2235130112, 0.0041821561, 0.095260223],
                         [0.7132692308, 0.1901923077, 0.0140384615, 0.0825],
                         [0.6737341772, 0.1914556962, 0.0053797468, 0.1294303797],
                         [0.6797082228, 0.2102122016, 0.0086206897, 0.1014588859],
                         [0.7419168591, 0.1345265589, 0.0005773672, 0.1229792148],
                         [0.7131226054, 0.186302682, 0.0138888889, 0.0866858238],
                         [0.6795039164, 0.1651436031, 0.0006527415, 0.1546997389],
                         [0.6756329114, 0.18407173, 0.0089662447, 0.1313291139]
                        ])

Pot_prob = torch.tensor([[0.100877193, 0.760025063, 0.128446115, 0.010651629],
                         [0.139466546, 0.784132007, 0.058996383, 0.017405063],
                         [0.123141264, 0.807156134, 0.059944238, 0.009758364],
                         [0.126346154, 0.786346154, 0.080961538, 0.006346154],
                         [0.131962025, 0.783860759, 0.078797468, 0.005379747],
                         [0.186339523, 0.711538462, 0.082891247, 0.019230769],
                         [0.213048499, 0.737297921, 0.037528868, 0.012124711],
                         [0.121168582, 0.749521073, 0.115421456, 0.013888889],
                         [0.214751958, 0.716057441, 0.058093995, 0.011096606],
                         [0.125, 0.785337553, 0.087025316, 0.002637131]
                        ])

Y1_prob = torch.tensor([0.07796727, 0.07891634, 0.09283598, 0.09125421, 0.09267781, 0.08207989, 0.09552501, 0.09884674 0.10011216 0.10185212 0.08793247])

T1_prob = torch.tensor([])